In [158]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [159]:
data = pd.read_csv('Resources/COVID19_2020_open_line_list - outside_Hubei.csv')
data2 = pd.read_csv('Resources/ebola_2014_2016_clean.csv')

In [160]:
data.columns

Index(['ID', 'age', 'sex', 'city', 'province', 'country',
       'wuhan(0)_not_wuhan(1)', 'latitude', 'longitude', 'geo_resolution',
       'date_onset_symptoms', 'date_admission_hospital', 'date_confirmation',
       'symptoms', 'lives_in_Wuhan', 'travel_history_dates',
       'travel_history_location', 'reported_market_exposure',
       'additional_information', 'chronic_disease_binary', 'chronic_disease',
       'source', 'sequence_available', 'outcome', 'date_death_or_discharge',
       'notes_for_discussion', 'location', 'admin3', 'admin2', 'admin1',
       'country_new', 'admin_id', 'data_moderator_initials'],
      dtype='object')

In [161]:
len(data.columns)

33

In [162]:
data_USA = data['country']=='United States'

united_states_filter = data [data_USA]
print(united_states_filter.shape)

(1212, 33)


In [163]:
united_states_filter

,ID,age,sex,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,...,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials
636,668,60,female,Chicago,Illinois,United States,1.0,41.878100,-87.629800,point,...,NaN,NaN,NaN,Chicago,NaN,NaN,Illinois,United States,14,NaN
637,669,35,male,Snohomish County,Washington,United States,1.0,48.048180,-121.696000,admin2,...,NaN,NaN,NaN,NaN,NaN,Snohomish County,Washington,United States,2988,NaN
645,677,50,male,Orange County,California,United States,1.0,33.703290,-117.761000,admin2,...,NaN,NaN,https://laist.com/2020/01/26/coronavirus-orang...,NaN,NaN,Orange County,California,United States,213,NaN
1033,1065,NaN,NaN,Los Angeles,California,United States,1.0,34.050000,-118.250000,point,...,NaN,NaN,Most details from listening to LA County Publi...,Los Angeles,NaN,NaN,California,United States,5,NaN
1034,1066,NaN,NaN,"Tempe, Maricopa County",Arizona,United States,1.0,33.412780,-111.943000,point,...,NaN,NaN,NaN,Tempe,NaN,Maricopa County,Arizona,United States,101,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20117,20569,NaN,NaN,NaN,New York,United States,1.0,43.014087,-75.646457,admin1,...,NaN,NaN,NaN,NaN,NaN,NaN,New York,United States,33,NaN
20118,20570,NaN,NaN,NaN,New York,United States,1.0,43.014087,-75.646457,admin1,...,NaN,NaN,NaN,NaN,NaN,NaN,New York,United States,33,NaN
20119,20571,NaN,NaN,NaN,New York,United States,1.0,43.014087,-75.646457,admin1,...,NaN,NaN,NaN,NaN,NaN,NaN,New York,United States,33,NaN
20120,20572,NaN,NaN,NaN,New York,United States,1.0,43.014087,-75.646457,admin1,...,NaN,NaN,NaN,NaN,NaN,NaN,New York,United States,33,NaN


In [164]:
united_states_filter = united_states_filter.drop(columns = ['wuhan(0)_not_wuhan(1)', 'geo_resolution', 'notes_for_discussion', 'admin3', 'admin2', 'admin_id', 'data_moderator_initials', 'sequence_available', 'country_new'])

In [165]:
united_states_filter = united_states_filter.set_index('ID')

In [166]:
united_states_filter['date_onset_symptoms'] = pd.to_datetime(united_states_filter['date_onset_symptoms'])

united_states_filter.head(1)

,age,sex,city,province,country,latitude,longitude,date_onset_symptoms,date_admission_hospital,date_confirmation,...,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,outcome,date_death_or_discharge,location,admin1
ID,,,,,,,,,,,,,,,,,,,,,
668,60,female,Chicago,Illinois,United States,41.8781,-87.6298,2020-01-16,NaN,24.01.2020,...,Wuhan,NaN,traveled to Wuhan in Dec; returned to U.S. Jan...,NaN,NaN,https://www.pbs.org/newshour/health/cdc-confir...,NaN,NaN,Chicago,Illinois


In [167]:
location=united_states_filter[['city','location','admin1','province','country','latitude','longitude']].copy()
df=united_states_filter.drop(columns=['city','location','admin1','province','country','latitude','longitude'])
df.count()

age                          299
sex                          301
date_onset_symptoms           13
date_admission_hospital       13
date_confirmation           1205
symptoms                      11
lives_in_Wuhan                 9
travel_history_dates          13
travel_history_location       87
reported_market_exposure       2
additional_information       272
chronic_disease_binary         9
chronic_disease                1
source                      1207
outcome                       20
date_death_or_discharge       18
dtype: int64

In [168]:
united_states_filter['date_confirmation'] = pd.to_datetime(united_states_filter['date_confirmation'])

united_states_filter

,age,sex,city,province,country,latitude,longitude,date_onset_symptoms,date_admission_hospital,date_confirmation,...,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,outcome,date_death_or_discharge,location,admin1
ID,,,,,,,,,,,,,,,,,,,,,
668,60,female,Chicago,Illinois,United States,41.878100,-87.629800,2020-01-16,NaN,2020-01-24,...,Wuhan,NaN,traveled to Wuhan in Dec; returned to U.S. Jan...,NaN,NaN,https://www.pbs.org/newshour/health/cdc-confir...,NaN,NaN,Chicago,Illinois
669,35,male,Snohomish County,Washington,United States,48.048180,-121.696000,2020-01-16,19.01.2020,2020-01-20,...,Wuhan,no,(1) https://www.nejm.org/doi/full/10.1056/NEJM...,1.0,hypertriglyceridemia,https://www.cdc.gov/media/releases/2020/p0121-...,NaN,NaN,NaN,Washington
677,50,male,Orange County,California,United States,33.703290,-117.761000,NaT,NaN,2020-01-26,...,Wuhan,NaN,NaN,NaN,NaN,https://www.cnbc.com/2020/01/26/third-us-case-...,NaN,NaN,NaN,California
1065,NaN,NaN,Los Angeles,California,United States,34.050000,-118.250000,2020-01-22,22.01.2020,2020-01-26,...,Wuhan,NaN,"From Wuhan, went through LAX to a vacation (po...",NaN,NaN,https://www.latimes.com/california/story/2020-...,NaN,NaN,Los Angeles,California
1066,NaN,NaN,"Tempe, Maricopa County",Arizona,United States,33.412780,-111.943000,NaT,NaN,2020-01-26,...,Wuhan,NaN,Arizona State University community member; rec...,NaN,NaN,https://azdhs.gov/director/public-information-...,NaN,NaN,Tempe,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20569,NaN,NaN,NaN,New York,United States,43.014087,-75.646457,NaT,NaN,2020-12-03,...,NaN,NaN,NaN,NaN,NaN,https://www.health.ny.gov/diseases/communicabl...,NaN,NaN,NaN,New York
20570,NaN,NaN,NaN,New York,United States,43.014087,-75.646457,NaT,NaN,2020-12-03,...,NaN,NaN,NaN,NaN,NaN,https://www.health.ny.gov/diseases/communicabl...,NaN,NaN,NaN,New York
20571,NaN,NaN,NaN,New York,United States,43.014087,-75.646457,NaT,NaN,2020-12-03,...,NaN,NaN,NaN,NaN,NaN,https://www.health.ny.gov/diseases/communicabl...,NaN,NaN,NaN,New York


In [169]:
data2

,Country,Date,No. of suspected cases,No. of probable cases,No. of confirmed cases,"No. of confirmed, probable and suspected cases",No. of suspected deaths,No. of probable deaths,No. of confirmed deaths,"No. of confirmed, probable and suspected deaths"
0,Guinea,2014-08-29,25.0,141.0,482.0,648.0,2.0,141.0,287.0,430.0
1,Nigeria,2014-08-29,3.0,1.0,15.0,19.0,0.0,1.0,6.0,7.0
2,Sierra Leone,2014-08-29,54.0,37.0,935.0,1026.0,8.0,34.0,380.0,422.0
3,Liberia,2014-08-29,382.0,674.0,322.0,1378.0,168.0,301.0,225.0,694.0
4,Sierra Leone,2014-09-05,78.0,37.0,1146.0,1261.0,11.0,37.0,443.0,491.0
...,...,...,...,...,...,...,...,...,...,...
2480,Liberia,2016-03-23,5636.0,1879.0,3151.0,10666.0,NaN,NaN,NaN,4806.0
2481,Italy,2016-03-23,0.0,0.0,1.0,1.0,NaN,NaN,NaN,0.0
2482,Liberia,2016-03-23,0.0,3.0,2.0,5.0,NaN,3.0,1.0,4.0
2483,Nigeria,2016-03-23,0.0,1.0,19.0,20.0,0.0,1.0,7.0,8.0


In [170]:
data2_USA = data2['Country']=='United States of America'

united_states_filter_2 = data2[data2_USA]
print(united_states_filter_2.shape)

(245, 10)


In [171]:
united_states_filter_2 
united_states_filter_2 = united_states_filter_2.drop(columns = ['No. of confirmed, probable and suspected cases', 'No. of suspected deaths', 'No. of probable deaths', 'No. of confirmed, probable and suspected deaths'])

united_states_filter_2 

,Country,Date,No. of suspected cases,No. of probable cases,No. of confirmed cases,No. of confirmed deaths
49,United States of America,2014-10-03,NaN,NaN,1.0,0.0
56,United States of America,2014-10-08,NaN,NaN,1.0,0.0
61,United States of America,2014-10-10,NaN,NaN,1.0,0.0
68,United States of America,2014-10-15,NaN,NaN,2.0,1.0
79,United States of America,2014-10-17,NaN,NaN,3.0,1.0
...,...,...,...,...,...,...
2439,United States of America,2015-12-17,0.0,0.0,4.0,1.0
2449,United States of America,2015-12-22,0.0,0.0,4.0,1.0
2461,United States of America,2015-12-23,0.0,0.0,4.0,1.0
2470,United States of America,2015-12-29,0.0,0.0,4.0,1.0


In [172]:
united_states_filter_2 = united_states_filter_2.rename(columns={'No. of suspected cases': 'Suspected Cases', 'No. of probable cases': 'Probable Cases', 'No. confirmed cases': 'Confirmed Cases', 'No. of suspected deaths': 'Suspected Deaths'})
united_states_filter_2

,Country,Date,Suspected Cases,Probable Cases,No. of confirmed cases,No. of confirmed deaths
49,United States of America,2014-10-03,NaN,NaN,1.0,0.0
56,United States of America,2014-10-08,NaN,NaN,1.0,0.0
61,United States of America,2014-10-10,NaN,NaN,1.0,0.0
68,United States of America,2014-10-15,NaN,NaN,2.0,1.0
79,United States of America,2014-10-17,NaN,NaN,3.0,1.0
...,...,...,...,...,...,...
2439,United States of America,2015-12-17,0.0,0.0,4.0,1.0
2449,United States of America,2015-12-22,0.0,0.0,4.0,1.0
2461,United States of America,2015-12-23,0.0,0.0,4.0,1.0
2470,United States of America,2015-12-29,0.0,0.0,4.0,1.0


In [173]:
rds_connection_string = "postgres:postgres@localhost:5432/pandemic_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [176]:
united_states_filter.to_sql(name='Coronavirus', con=engine, if_exists='append', index=False)

In [177]:
united_states_filter_2.to_sql(name='Ebola', con=engine, if_exists='append', index=False)